<a href="https://colab.research.google.com/github/habrev/Rossmann-Pharmaceuticals/blob/task-2/notebooks/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependency

In [1]:
import pandas as pd
import matplotlib as plt

# Preprocessing

# file imported

In [2]:
df = pd.read_csv('train.csv')

<ipython-input-2-cd75a77447fc>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


In [3]:
from sklearn.preprocessing import LabelEncoder

# Display the first few rows of the dataset
print("Original Data:")
print(df.head())

# Identify non-numerical columns
non_numerical_columns = df.select_dtypes(include=['object', 'category']).columns

# Convert all non-numerical columns to strings (if necessary)
df[non_numerical_columns] = df[non_numerical_columns].astype(str)

# Perform One-Hot Encoding on all non-numerical columns
df_encoded = pd.get_dummies(df, columns=non_numerical_columns)

# Optionally, save the encoded data to a new file
output_path = 'one_hot_encoded.csv'
df_encoded.to_csv(output_path, index=False)
print(f"\nEncoded file saved to {output_path}")

print("\nEncoded Data (sample rows for 'Date' and 'StateHoliday'):")
encoded_columns = [col for col in df_encoded.columns if 'Date' in col or 'StateHoliday' in col]
print(df_encoded[encoded_columns].head())


Original Data:
   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday  
0              1  
1              1  
2              1  
3              1  
4              1  

Encoded file saved to one_hot_encoded.csv

Encoded Data (sample rows for 'Date' and 'StateHoliday'):
   Date_2013-01-01  Date_2013-01-02  Date_2013-01-03  Date_2013-01-04  \
0            False            False            False            False   
1            False            False            False            False   
2            False            False            False            False   
3 

# feature engineering

In [4]:
df = pd.read_csv('one_hot_encoded.csv')

In [7]:
from sklearn.preprocessing import StandardScaler
# Identify numerical columns to scale
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Initialize the scaler
scaler = StandardScaler()

# Scale the numerical columns
df_scaled = df.copy()
df_scaled[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Save the scaled dataset
scaled_output_path = 'scaled_dataset.csv'
df_scaled.to_csv(scaled_output_path, index=False)

# Display a preview of the scaled DataFrame
print(f"Scaled Data saved to {scaled_output_path}")
print(df_scaled.head())

Scaled Data saved to scaled_dataset.csv
      Store  DayOfWeek     Sales  Customers      Open     Promo  \
0 -1.731640   0.501484 -0.132683  -0.168269  0.452399  1.273237   
1 -1.728534   0.501484  0.075373  -0.017540  0.452399  1.273237   
2 -1.725427   0.501484  0.659800   0.404499  0.452399  1.273237   
3 -1.722321   0.501484  2.135414   1.862258  0.452399  1.273237   
4 -1.719214   0.501484 -0.247231  -0.159656  0.452399  1.273237   

   SchoolHoliday  Date_2013-01-01  Date_2013-01-02  Date_2013-01-03  ...  \
0       2.144211            False            False            False  ...   
1       2.144211            False            False            False  ...   
2       2.144211            False            False            False  ...   
3       2.144211            False            False            False  ...   
4       2.144211            False            False            False  ...   

   Date_2015-07-26  Date_2015-07-27  Date_2015-07-28  Date_2015-07-29  \
0            False         